# Description

In [2]:
# libraries
library(Seurat)
library(tidyverse)
library(igraph)
require(circlize)
library(R.utils)
library(data.table) #to read gz file

Warning message:
“package ‘Seurat’ was built under R version 4.2.2”
Attaching SeuratObject

Warning message:
“package ‘tidyverse’ was built under R version 4.2.1”
── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
Warning message:
“package ‘stringr’ was built under R version 4.2.1”
Warning message:
“package ‘forcats’ was built under R version 4.2.1”
── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
“package ‘igraph’ was built under R version 4.2.1”

A

### Read in the expression data of interacting cells:
The dataset used here is publicly available single-cell data from XXX. The data was processed, and filtered by applying XXX. 

In [5]:
getwd()

[1] "/work/project/ladcol_011/clone_GH/community-paper/src/method_comparison/compare_algorithms/run_CellPhoneDB/run_algorithm"

In [13]:
input_dir <- "../../../../../results/data_preprocessing/Lasry/preprocessed/"
output_dir <- "../../../../../results/method_comparison/compare_algorithms/CPDB/"
final_out <- "../../../../../results/method_comparison/compare_results/CPDB/"

In [8]:
# # load counts
# print("load counts")
# counts <- read.table(gzfile(paste0(path_in,"/counts_corr.csv.gz")
#                             )
#                      ,sep = ","
#                      ,row.names = 1
#                      ,header = TRUE
#                      )
# # load counts

counts <- fread(paste0(input_dir,"counts_corr.csv.gz"), header = TRUE,check.names=FALSE)
counts <- as.data.frame(counts)
rownames(counts) <- counts$gene_symbol
counts <- counts[,-1]
# head(str(counts))
# print(str(counts))

In [9]:
# load cell annotation
print("load cell annotation")
anno_cells <- read.table(paste0(input_dir,"anno_cells_corr.txt")
                         ,sep = "\t"
                         ,row.names = 1
                         ,header = TRUE
                         ,check.names=FALSE
                         )
# print(str(anno_cells))

[1] "load cell annotation"


In [10]:
#set rownames of annotation to cell_ids
rownames(anno_cells) <- anno_cells$cell

In [11]:
#set colnames of counts to cell_ids
colnames(counts) <- rownames(anno_cells)

In [12]:
#create a Seurat object
srt=CreateSeuratObject(counts=counts, meta.data=anno_cells)

In [13]:
#peek into the number of cells for case/control
srt@meta.data$health_status %>% table()

.
    AML healthy 
  21311   25391 

In [14]:
#peek into the number of cell types
srt@meta.data$cell_type %>% table()

.
    B    DC   Ery  Gran  HSPC  Mono    NK     T 
 4765  1634  1674  2332  3169 18004  3078 12046 

In [15]:
#set the indent to cell_type
Idents(srt) <- "cell_type"

In [16]:
# initialize empty vector for storing DEGs
DEGs <- c()

# iterate over each unique cell type 
for (cell in unique(srt@meta.data$cell_type)) {
  
  # subset Seurat object to only include cells of current cell type
  seurat_obj_receiver <- subset(srt, idents = cell)
  
  # set cell identity using the "health_status" feature
  seurat_obj_receiver <- SetIdent(seurat_obj_receiver, value = seurat_obj_receiver[["health_status"]])
  
  # specify the two conditions to compare
  condition_oi <- "AML"
  condition_reference <- "healthy" 
  
  # find differentially expressed genes between the two conditions
  DE_table_receiver <- FindMarkers(object = seurat_obj_receiver, 
                                   ident.1 = condition_oi, 
                                   ident.2 = condition_reference, 
                                   min.pct = 0.10) %>%
    # convert row names to a separate "gene" column
    rownames_to_column("gene")
  
  # add cell type information to the DEG table
  DE_table_receiver <- data.frame(cluster = cell, DE_table_receiver)
  
  # filter DEGs based on statistical significance and fold change threshold
  DE_table_receiver <- DE_table_receiver %>% 
    filter(p_val_adj <= 0.05 & abs(avg_log2FC) >= 0.25)
  
  # print cell type and number of DEGs found
  print(cell)
  print(nrow(DE_table_receiver))
  
  # append DEGs to the vector of all DEGs
  DEGs <- rbind(DEGs, DE_table_receiver)
}


[1] "Mono"
[1] 178
[1] "Gran"
[1] 103
[1] "T"
[1] 48
[1] "NK"
[1] 164
[1] "B"
[1] 74
[1] "HSPC"
[1] 90
[1] "Ery"
[1] 645
[1] "DC"
[1] 56


In [26]:
# write.table(DEGs, file =paste0(output_dir,"samples_DEGs/DEGs.tsv"), sep = '\t', quote = F, row.names = F)

In [18]:
meta <- anno_cells["cell_type"] %>% rownames_to_column("Cell")

Below code takes an expression counts matrix (counts) and an annotation data frame (anno_cells) and writes out a separate counts file and metadata file for each sample ID in the sample_ID column of anno_cells. Each metadata file contains a single column (cell_type) and a row for each cell in the sample (required by CellPhoneDB), while the counts file contains the expression counts for each gene in each cell.

In [19]:
# create a directory "samples_DEGs" to save the subsetted counts and annotation files. 
dir.create(file.path(output_dir, "samples_DEGs"))

# loop over each unique sample ID in the "sample_ID" column of the "anno_cells" data frame
for (sample in unique(anno_cells$sample_ID)) {
  
  # filter the annotation data frame to include only cells from the current sample
  anno_filtered <- filter(anno_cells, sample_ID == sample)
  
  # subset the expression counts matrix to the current sample
  subset_counts <- counts[, rownames(anno_filtered)]
  
  # subset the annotation data frame (required by CellPhoneDB)
  subset_meta <- anno_filtered["cell_type"] %>% rownames_to_column("Cell")
    
  # subset DEGs
  subset_DEGs <- DEGs %>% filter(cluster %in% unique(subset_meta$cell_type))
  
  # write the subsetted annotation data frame to a tab-separated value (TSV) file
  write.table(subset_meta, paste0(output_dir,"samples_DEGs/", sample, "_meta.tsv"), sep = '\t', quote = F, row.names = F)
  
  # write the subsetted counts matrix to a TSV file
  write.table(subset_counts, paste0(output_dir,"samples_DEGs/", sample, "_counts.tsv"), sep = '\t', quote = F)

  write.table(subset_DEGs, paste0(output_dir,"samples_DEGs/", sample, "_DEGs.tsv"), sep = '\t', quote = F)



}


Warning message in dir.create(file.path(output_dir, "samples_DEGs")):
“'../../../../../results/method_comparison/compare_algorithms/CPDB//samples_DEGs' already exists”


Below is the content of shell script (`./runCPDB.sh`) that performs CellPhoneDB using DEG analysis method for each sample in the /samples_DEGs/ directory.

For each sample, the script creates a new directory `(${sample}_results)` to store the results of the CellPhoneDB analysis. The cellphonedb method degs_analysis command runs the DEG analysis method on the metadata and counts files for the current sample, using the `../DEGs.tsv` file as input for the list of differentially expressed genes. The `--database` option specifies the path to the CellPhoneDB database to use for the analysis, while the `--counts-data` option specifies the type of gene identifier used in the counts file (in this case, `hgnc_symbol`). The `--output-path` option specifies the directory where the analysis results will be saved.

`'./runCPDB.sh'`

```bash
# Set the directory path to the directory containing the DEG samples
samples_dir=../../../../../results/method_comparison/compare_algorithms/CPDB/samples_DEGs/

# Get a list of sample names
my_vars=$(ls "$samples_dir" | cut -d_ -f1 | uniq)

# Set the path to the custom database file
custom_db=../../../../../results/method_comparison/build_customDB/CPDB/custom_cellphone.db

# Loop over each sample variable name
for sample in $my_vars;
do
  # Create a subdirectory for the sample results
  mkdir ${samples_dir}${sample}_results;

  # Run CellPhoneDB's DEG analysis method on the sample using the custom database, with input files in the sample directory and output files in the sample results subdirectory
  cellphonedb method degs_analysis ${samples_dir}${sample}_meta.tsv ${samples_dir}${sample}_counts.tsv ${samples_dir}${sample}_DEGs.tsv --database $custom_db --counts-data hgnc_symbol --output-path ${samples_dir}${sample}_results/;
done;
```

In [7]:
run_CPDB <- './runCPDB.sh'

In [8]:
system(run_CPDB)

### Restructure CellPhoneDB's outputs

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# !!!!!!!!!!!!!!!!!!!we gotta explain why we are using means file not the significant ones

In [9]:
results_dir <- list.dirs(path = paste0(output_dir,"samples_DEGs/"), full.names = TRUE)

In [10]:
results_dir <- results_dir[grepl("_results", results_dir, fixed = TRUE)]

In [11]:
# Define a function called 'restructure_result' that takes one argument, 'cpdb_means'
restructure_result <- function(cpdb_means) {
  
  # Subset the columns of 'cpdb_means' that contain 'interacting_pair' or '|'
  cpdb_means <- cpdb_means[, grepl('interacting_pair|\\|', colnames(cpdb_means))]
  
  # Pivot the data to long format and split the 'interacting_pair' column into 'sending_protein' and 'receiving_protein' columns
  # Split the 'cell_types' column into 'sending_celltype' and 'receiving_celltype' columns
  # Unite the 'sending_celltype' and 'sending_protein' columns into a single column called 'sender'
  # Unite the 'receiving_celltype' and 'receiving_protein' columns into a single column called 'receiver'
  # Unite the 'sender' and 'receiver' columns into a single column called 'interacting_pairs'
  # Select the 'interacting_pairs' and 'value' columns
  conversion <- cpdb_means %>%
    pivot_longer(cols = -interacting_pair, names_to = "cell_types", values_to = "value") %>%
    separate(interacting_pair, c("sending_protein", "receiving_protein"), sep = "_") %>%
    separate(cell_types, c("sending_celltype", "receiving_celltype"), sep = "\\|") %>%
    unite(sender, c("sending_celltype", "sending_protein"), sep = ":", remove = FALSE) %>%
    unite(receiver, c("receiving_celltype", "receiving_protein"), sep = ":", remove = FALSE) %>%
    unite(interacting_pairs, c("sender", "receiver"), sep = "_", remove = FALSE) %>%
    select(interacting_pairs, value)
  
  # Return the processed data
  return(conversion)
}


In [12]:
results=list()
for (sample in results_dir){
    
    file <- paste0(sample,"/means.txt")
    
    sample_id <- basename(sample)
    sample_id <- strsplit(sample_id, '_')[[1]][1]
    
    
    if (file.exists(file)){
        
        cpdb_means <- read.csv(file, sep = "\t",  check.names = FALSE)
        
        
        sample_result <- restructure_result(cpdb_means)
        colnames(sample_result) <- c("interaction",sample_id)
        results[[sample_id]] <- sample_result
        
    }
    
}

In [15]:
# Define a variable called `result` that will hold the output of the Reduce function
matrix_result <- Reduce(
  
  # The `Reduce()` function takes two arguments: a function and a list.
  # In this case, the function is an anonymous function defined using the `function()` keyword.
  # This function takes two arguments `x` and `y` and performs a full join between them using the `full_join()` function from the `dplyr` package.
  # The `by = "interaction"` argument specifies that the join should be performed on the "interaction" column.
  function(x, y) full_join(x, y, by = "interaction"), 
  
  # The second argument to the `Reduce()` function is a list called `results`.
  # This list contains data frames that need to be joined together.
  results
)

In [16]:
head(matrix_result)

interaction,AML-0024,AML-0160,AML-0693,AML-1371,AML-2123,AML-3133,AML-4340,healthy-1,healthy-2,healthy-3,healthy-4,healthy-4003,healthy-5
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
B:GNAS_B:ADRB2,0.438,0.000,0.369,0.488,0.000,0.000,0.518,0.457,0.524,0.461,0.450,0.353,0.350
B:GNAS_DC:ADRB2,0.000,0.579,0.378,0.577,NA,0.000,0.517,0.484,0.536,0.466,0.485,0.368,0.356
B:GNAS_Ery:ADRB2,0.446,0.000,0.000,0.000,0.524,0.000,NA,0.000,0.000,0.000,0.000,0.000,0.350
B:GNAS_Gran:ADRB2,0.444,0.586,0.392,0.601,0.000,0.000,0.000,0.478,0.546,0.480,0.469,0.390,0.365
B:GNAS_HSPC:ADRB2,0.000,0.000,0.374,0.581,0.524,0.427,0.517,0.458,0.521,0.462,0.445,0.369,0.357
B:GNAS_Mono:ADRB2,0.451,0.588,0.385,0.529,0.000,0.432,0.517,0.477,0.536,0.466,0.463,0.356,0.355


In [17]:
write.csv(matrix_result, paste0(final_out,"CPDB_results.csv"))

In [ ]:
getwd()